In [1]:
from google.colab import drive
drive.mount('/content/drive')
! pip install tensorboardX

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
     |████████████████████████████████| 225kB 2.8MB/s 


## Данные

Был проведён визуальный анализ данных, который показал следующее.

Обучающая выборка представляет собой набор трёхканальных изображений размера 320x240 количеством около тысячи, что очень мало для решения подобного рода задач. На большинстве из них запечатлены только лица людей, но присутствуют также фотографии человека в полный рост. Почти все фотографии цветные, но встречаются и чёрно-белые. Люди на фотографиях - разных рас, пола, возраста, имеют разное выражение лица и стоят в разных позах.

На фотографиях человек занимает большую часть площади, в большинстве случаев - больше половины. 

## Аугментация

Поскольку количество данных чрезвычайно мало, для повышения качества модели необходимо использовать аугментацию.

В ходе решения были рассмотрены следующие методы:

1) отражения относительно горизонтальной и вертикальной осей,

2) обрезание с последующим растяжением или сдвигом относительно первоначального положения,

3) вращение относительно произвольного центра на произвольное число градусов,

4) цветовая аугментация (изменение яркости/контрастности, обмен местами каналов изображения, обращение изображения, изменение яркости каналов, преобразование в чёрно-белое изображение),

5) наложение шума 

## Выбор модели

Существует несколько методов решения задачи сегментации.

1) patch-based: обучаем сетку классифицировать кусочки изображения как объекты или фон, затем комбинируем ответы для получения итоговой маски

2) энкодер: сжимаем, а затем разжимаем изображение, получая на выходе маску (SegNet, Unet)

3) алгоритмы компьютерного зрения для распознавания лиц

В контексте рассматриваемой задачи наименее трудозатратным методом является использование нейросети-энкодера. Жизненный опыт подсказывал, что лучше Unet ничего не работало... 

В качестве основной модели использовался Unet, более простая модель энкодера работала хуже. Параметры сети выбирались эмпирически (batch_size, learning_rate и т.д. ). Рассматривались модели с разным количеством фильтров и глубиной. 

Были рассмотрены различные подходы к аугментации данных. Из применённых методов не слишком эффективными оказались цветовые, а вот вращение и обрезание с последующим растяжением ощутимо улучшили устойчивость модели.

Вообще говоря, для решения задачи можно было бы использовать предобученную модель (например, от Google) для сегментации человека, но это было бы нечестно в контексте данного задания.

In [2]:
! python3 "/content/drive/My Drive/laba/train.py"

Current epoch:  0
100% 263/263 [06:28<00:00,  1.95s/it]
Epoch loss 0.07502820243626947
Epoch dice 0.6573579757604269
Make test
  0% 0/145 [00:00<?, ?it/s]/content/drive/My Drive/laba/train.py:86: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  cur_input = Variable(cur_input, volatile = True)
/content/drive/My Drive/laba/train.py:87: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  cur_target = Variable(cur_target, volatile = True)
100% 145/145 [00:39<00:00,  3.65it/s]
Test loss 0.32820962338612
Test dice 0.6720700833078368
Current epoch:  1
100% 263/263 [02:17<00:00,  2.35it/s]
Epoch loss 0.05418762400916321
Epoch dice 0.7443959247694905
Make test
100% 145/145 [00:04<00:00, 31.64it/s]
Test loss 0.2354416797901022
Test dice 0.7651522052860025
Current epoch:  2
100% 263/263 [02:17<00:00,  2.37it/s]
Epoch loss 0.04879469057339679
Epoch dice 0.7714197339104552
Make test
100% 145/145 [00:04<00:00,

## Загрузка модели


In [0]:
%cd "/content/drive/My Drive/laba/"

In [3]:
import torch
from model import SegmenterModel
from metrics import *
from image_dataset import *
import torch
import torch.nn as nn
import torch.utils.data as dt
from torch.autograd import Variable
import torch.optim as optim
import matplotlib.pyplot as plt
import os
from tqdm import *
import numpy as np
import warnings
warnings.filterwarnings('ignore')

segm_model = SegmenterModel()
segm_model.load_state_dict(torch.load('/content/drive/My Drive/laba/segm_model1.pth', map_location=torch.device('cpu')))
segm_model.eval()

SegmenterModel(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
  )
  (conv3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
  )
  (mp2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
  )
  (mp3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv7): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_run

## Генерация масок для validate

In [28]:

from google.colab.patches import cv2_imshow
from skimage.morphology import closing
from html import *
from utils import *

seg_model = segm_model.cuda().eval()
test = '/content/drive/My Drive/laba/data/valid/'
test_masks = '/content/drive/My Drive/laba/data/valid_mask'

rle_masks = []
masks = []

ds_test = ImageDataset(test, test_masks, augment=False)
dl_test = dt.DataLoader(ds_test, shuffle=False, num_workers=4, batch_size=1)
dice = 0
for iter, (i, t) in enumerate(tqdm(dl_test)):
    i = Variable(i, volatile = True)
    t = Variable(t, volatile = True)
    o = segm_model(i.cuda()).cpu()
    
    o = (o >= 0.5).type(torch.float)
    pict = np.array(o[0,0])
    mask = np.array(t[0,0])
    dice = (2 * np.sum(pict * mask)) / (np.sum(pict) + np.sum(mask)) 
    rle_masks.append(encode_rle(mask))
    masks.append(mask)
    
print(len(masks))    

    

100%|██████████| 145/145 [00:04<00:00, 30.28it/s]

145


In [0]:
import pandas as pd

da = pd.DataFrame(masks)
da.columns = ['rle_mask']
da["id"] = np.arange(1315, 1460)
da.to_csv('/content/drive/My Drive/laba/pred_valid_template.csv', index=False)

## Генерация HTML-страницы


In [7]:
from glob import glob
seg_model = segm_model.cuda().eval()
test = '/content/drive/My Drive/laba/data/test/'

seg_model = segm_model.cuda().eval()

rle_masks = []
masks = []

ds_test = OnlyImageDataset(test)
dl_test = dt.DataLoader(ds_test, shuffle=False, num_workers=4, batch_size=1)
dice = 0
for iter, i in enumerate(tqdm(dl_test)):
    i = Variable(i, volatile = True)
    o = segm_model(i.cuda()).cpu()
    
    o = (o >= 0.5).type(torch.float)
    pict = (255*np.array(o[0,0])).astype(np.uint8)
    masks.append(pict)
    
print(len(masks)) 

from html import *
paths_to_imgs = sorted(glob("/content/drive/My Drive/laba/data/test/*"))
pred_masks = masks

_ = get_html(paths_to_imgs, pred_masks, path_to_save="/content/drive/My Drive/laba/results/example")

100%|██████████| 100/100 [00:03<00:00, 34.64it/s]

100
